In [2]:
import pandas as pd

file_path = 'Content_Reco.xlsx'
user_df = pd.read_excel(file_path, sheet_name='user')
physio_df = pd.read_excel(file_path, sheet_name='physio')

# Content Matching

calculation logic

Select top50 matching trainers for each user based on match_score

In [3]:
# Add temporary key for Cartesian product
user_df['key'] = 1
physio_df['key'] = 1
full_df = pd.merge(user_df, physio_df, on='key').drop('key', axis=1)

# Define specialties intersection score function
def specialties_score(user_spec, trainer_spec):
    user_set = set([s.strip().lower() for s in str(user_spec).split(',')])
    trainer_set = set([s.strip().lower() for s in str(trainer_spec).split(',')])
    n = len(user_set & trainer_set)
    return min(n, 3)

# Add age_group matching
def compute_row_score(row):
    score = 0
    score += 1 if row['gender_x'] == row['gender_y'] else 0
    score += 1 if row['city_x'] == row['city_y'] else 0
    score += 1 if row['available_time_x'] == row['available_time_y'] else 0
    score += specialties_score(row['preferred_specialties'], row['specialities'])
    score += 1 if row['age_group'] == row['serve_age_group'] else 0  # newly added
    return score

# Batch score calculation (maximum 7 points)
full_df['raw_score'] = full_df.apply(compute_row_score, axis=1)
full_df['match_score'] = full_df['raw_score'] / 7.0

# Extract and rename required fields
result_df = full_df[[
    'user_id', 'trainer_id', 'name', 'match_score', 'raw_score', 'specialities',
    'city_y', 'gender_y', 'available_time_y', 'age_group', 'serve_age_group'
]].rename(columns={
    'name': 'trainer_name',
    'city_y': 'city',
    'gender_y': 'gender',
    'available_time_y': 'available_time'
})

# Group-wise Top50 selection
result_df['rank'] = result_df.groupby('user_id')['match_score'].rank(method='first', ascending=False)
top50_df = result_df[result_df['rank'] <= 50].copy()
top50_df = top50_df.sort_values(['user_id', 'match_score'], ascending=[True, False])

# Output
output_path = 'user_top50_trainers_with_age.xlsx'
top50_df.to_excel(output_path, index=False)
output_path


'user_top50_trainers_with_age.xlsx'

# 2.Further Filtering

Top10 users filtered based on comprehensive ranking and content filtering Top50 (for cold-start users without any behavioural data)

Further filter the top10 in the candidate pool for recommendation

In [4]:
import pandas as pd
import numpy as np

file_path = 'Content_Reco.xlsx'
df_top50 = pd.read_excel(file_path, sheet_name='top50_match_rank')
df_a = pd.read_excel(file_path, sheet_name='GroupA_Rank')
df_b = pd.read_excel(file_path, sheet_name='GroupB_Rank')
df_c = pd.read_excel(file_path, sheet_name='GroupC_Rank')

# Assume final_score exists in one of A/B/C group tables, merge them to get complete trainer_id-final_score mapping
df_a['group'] = 'A'
df_b['group'] = 'B'
df_c['group'] = 'C'
trainer_info = pd.concat([df_a[['trainer_id', 'final_score', 'group']],
                          df_b[['trainer_id', 'final_score', 'group']],
                          df_c[['trainer_id', 'final_score', 'group']]], axis=0, ignore_index=True)

# 2. Merge final_score and group info into top50 table
df = df_top50.merge(trainer_info, on='trainer_id', how='left')

# 3. Normalize match_score and final_score (normalize across all trainers to avoid weight bias)
# Here we assume match_score is already in [0,1]; otherwise normalize
if df['match_score'].max() > 1.01 or df['match_score'].min() < -0.01:
    ms_min, ms_max = df['match_score'].min(), df['match_score'].max()
    df['match_score'] = (df['match_score'] - ms_min) / (ms_max - ms_min)

fs_min, fs_max = trainer_info['final_score'].min(), trainer_info['final_score'].max()
df['final_score_norm'] = (df['final_score'] - fs_min) / (fs_max - fs_min)

# 4. Compute cold-start score
df['cold_start_score'] = 0.5 * df['match_score'] + 0.5 * df['final_score_norm']

# 5. Select TopN per user by group: A4 B3 C3; if insufficient, fill with others
result_list = []
for user_id, user_df in df.groupby('user_id'):
    part_a = user_df[user_df['group']=='A'].sort_values('cold_start_score', ascending=False).head(4)
    part_b = user_df[user_df['group']=='B'].sort_values('cold_start_score', ascending=False).head(3)
    part_c = user_df[user_df['group']=='C'].sort_values('cold_start_score', ascending=False).head(3)
    frames = [part_a, part_b, part_c]
    res = pd.concat(frames)
    # If total less than 10, fill with the remaining highest cold_start_score trainers
    if res.shape[0] < 10:
        others = user_df[~user_df['trainer_id'].isin(res['trainer_id'])]
        needed = 10 - res.shape[0]
        res = pd.concat([res, others.sort_values('cold_start_score', ascending=False).head(needed)])
    # Sort and assign rank
    res = res.sort_values('cold_start_score', ascending=False).head(10)
    res['recommend_rank'] = np.arange(1, res.shape[0]+1)
    result_list.append(res)

final_df = pd.concat(result_list)

# 6. Keep/rename required columns and export
final_df = final_df[['user_id', 'trainer_id', 'match_score', 'final_score', 'cold_start_score', 'group', 'recommend_rank']]
final_df['match_score'] = final_df['match_score'].round(3)
final_df['final_score'] = final_df['final_score'].round(3)
final_df['cold_start_score'] = final_df['cold_start_score'].round(3)

output_path = 'cold_start_top10_by_group.xlsx'
final_df.to_excel(output_path, index=False)
output_path


'cold_start_top10_by_group.xlsx'